In [1]:
!python main_3d.py --dataset_dir /nfs/managed_datasets/LIDC-IDRI/ --scratch_dir /scratch/casparl/lidc-idri/

Copying files from /nfs/managed_datasets/LIDC-IDRI/tfrecords_4x4x4 to  /scratch/casparl/lidc-idri/tfrecords_4x4x4 
Done!
2019-09-04 17:52:24.607394: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-09-04 17:52:24.649815: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: TITAN RTX major: 7 minor: 5 memoryClockRate(GHz): 1.77
pciBusID: 0000:3b:00.0
2019-09-04 17:52:24.650448: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2019-09-04 17:52:24.652005: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2019-09-04 17:52:24.653425: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10.0
2019-09-04 17:52:24.654032: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Success

In [37]:
import ops_3d as ops
import tensorflow as tf
x_init = tf.random.uniform([20, 32, 32, 32, 1])


with tf.GradientTape() as tape:
    tape.watch(x)
    res=32
    res_next=16
    n_f_next=8
    n_f=1
    alpha=0.3
    r_res0=32
    x = ops.fromrgb(x_init,r_res0,n_f)
    print('Here1')
    print(x.shape)
    x = ops.discriminator_block(x,res,n_f,n_f_next)
    print('Here2')
    print(x.shape)

    x_init = ops.downscale3d(x_init)
    y = ops.fromrgb(x_init, res_next, n_f_next)
    x = ops.smooth_transition(y, x, res, r_res0, alpha)
    
    x = ops.discriminator_last_block(x,64,16,64)
    print('Here3')
    print(x.shape)
    z = tf.multiply(x,x)
    
dz_dx = tape.gradient(z,x_init)

Here1
(20, 32, 32, 32, 1)
(3, 3, 3, 1, 1)
(3, 3, 3, 1, 1)
(2, 2, 2, 8, 1)
(2, 2, 2, 8, 1)
Here2
(20, 16, 16, 16, 1)
(2, 2, 2, 1, 1)
(2, 2, 2, 1, 1)
Here3
(20, 1)


InvalidArgumentError: Conv3DBackpropInputOp: filter and out_backprop must have the same out_depth [Op:Conv3DBackpropInputV2]

In [11]:
# DOESNT WORK
import ops_3d as ops
import tensorflow as tf
x_init = tf.random.uniform([20, 32, 32, 32, 1])

with tf.GradientTape() as tape:
    tape.watch(x_init)
    res=32
    res_next=4
    n_f_next=4
    n_f=1
    alpha=0.3
    r_res0=16
    x = ops.fromrgb(x_init,r_res0,n_f)
    print('Here1')
    print(x.shape)
    #x = ops.discriminator_block(x,res,n_f,n_f_next)
    print('Here2')
    print(x.shape)

    x_init2 = ops.downscale3d(x_init)
    y = ops.fromrgb(x_init2, res_next, n_f_next)
    x = ops.smooth_transition(y, x, res, r_res0, alpha)
    
    x = ops.discriminator_last_block(x,res,n_f,n_f)
    print('Here3')
    print(x.shape)
    z = tf.multiply(x,x)
    
dz_dx = tape.gradient(z,x_init)

Here1
(20, 32, 32, 32, 1)
Here2
(20, 32, 32, 32, 1)
(2, 2, 2, 1, 1)
(2, 2, 2, 1, 1)


InvalidArgumentError: Incompatible shapes: [20,16,16,16,4] vs. [20,32,32,32,1] [Op:Sub] name: sub/

In [23]:
# WORKS
import ops_3d as ops
import tensorflow as tf
x_init = tf.random.uniform([20, 32, 32, 32, 1])

with tf.GradientTape() as tape:
    tape.watch(x_init)
    res=32
    res_next=4
    n_f_next=4
    n_f=1
    alpha=0.3
    r_res0=16
    x = ops.fromrgb(x_init,r_res0,n_f)
    print('Here1')
    print(x.shape)
    x = ops.discriminator_block(x,res,n_f,n_f_next)
    print('Here2')
    print(x.shape)

    x_init2 = ops.downscale3d(x_init)
    y = ops.fromrgb(x_init2, res_next, n_f_next)
    #x = ops.smooth_transition(y, x, res, r_res0, alpha)

    x = ops.discriminator_last_block(x,res,n_f,n_f)
    print('Here3')
    print(x.shape)
    z = tf.multiply(x,x)
    
dz_dx = tape.gradient(z,x_init)

Here1
(20, 32, 32, 32, 1)
(3, 3, 3, 1, 1)
(3, 3, 3, 1, 1)
(2, 2, 2, 4, 1)
(2, 2, 2, 4, 1)
Here2
(20, 16, 16, 16, 1)
(2, 2, 2, 1, 1)
(2, 2, 2, 1, 1)
Here3
(20, 1)


InvalidArgumentError: Conv3DBackpropInputOp: filter and out_backprop must have the same out_depth [Op:Conv3DBackpropInputV2]

In [21]:
import ops_3d as ops
import tensorflow as tf
x_init = tf.random.uniform([20, 64, 64, 64, 1])

with tf.GradientTape() as tape:
    tape.watch(x_init)
    res=128
    res_next=64
    n_f_next=16
    n_f=1
    alpha=0.3
    r_res0=128
    x = ops.fromrgb(x_init,r_res0,n_f)
    print('Here1')
    print(x.shape)
    x = ops.discriminator_block(x,res,n_f,n_f_next)
    print('Here2')
    print(x.shape)

    x_init = ops.downscale3d(x_init)
    y = ops.fromrgb(x_init, res_next, n_f_next)
    x = ops.smooth_transition(y, x, res, r_res0, alpha)
    
    x = ops.discriminator_last_block(x,res,n_f,n_f)
    print('Here3')
    print(x.shape)
    z = tf.multiply(x,x)
    
dz_dx = tape.gradient(z,x_init)

Here1
(20, 64, 64, 64, 1)
(3, 3, 3, 1, 1)
(3, 3, 3, 1, 1)
Here2
(20, 32, 32, 32, 16)
(2, 2, 2, 1, 1)
(2, 2, 2, 1, 1)
Here3
(20, 1)
(3, 3, 3, 1, 1)
(3, 3, 3, 1, 1)
